In [ ]:
import torch
import numpy as np
import tensorflow as tf
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, GPT2Tokenizer
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

class ImageCaptioning:
    def __init__(self):
        self.resnet_model = models.resnet50(pretrained=True)
        self.resnet_model.eval()
        
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        
        self.model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
        self.feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
    
    def extract_features(self, image_path):
        image = Image.open(image_path)
        image = self.transform(image).unsqueeze(0)
        
        with torch.no_grad():
            features = self.resnet_model(image)
        
        return features

    def generate_caption(self, image_path):
        image = Image.open(image_path)
        inputs = self.feature_extractor(images=image, return_tensors="pt").to(self.device)

        output_ids = self.model.generate(inputs["pixel_values"], max_length=16, num_beams=4, early_stopping=True)
        
        caption = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return caption

    def display_image_with_caption(self, image_path, caption):
        image = Image.open(image_path)
        plt.imshow(image)
        plt.axis('off')
        plt.title(caption)
        plt.show()

image_captioning = ImageCaptioning()
image_path = r"C:\Users\kanni\OneDrive\Desktop\codsoft tasks\task 3 image.jpg"

features = image_captioning.extract_features(image_path)
caption = image_captioning.generate_caption(image_path)
print("Generated Caption:", caption)

image_captioning.display_image_with_caption(image_path, caption)


C:\Users\kanni\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kanni\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
